# AI모델활용 5주차 (6~8)

## 5-6 OpenCV와 Yolo를 활용한 실시간 객체 탐지 서비스

- 웹캠을 통해서 실시간으로 들어오는 영상에 대한 탐지 서비스
- 사용 프로그램 OpenCV, YOLO, PyQt5

### PyQt5
- GUI 애플리케이션을 만들기 위한 강력한 라이브러리로, 각종 위젯을 사용하여 사용자 인터페이스를 구성가능하게 만들어준다
- ***QApplication:***GUI 애플리케이션의 시작점이자 메인 이벤트 루프를 관리하는 클래스(이게 없으면 다음을 진행 못함)
- ***QLabel:*** 텍스트 또는 이미지를 표시할 수 있는 위젯(텍스트 크기, 이미지 크기 조정 설성 가능)
- ***QVBoxLayout:*** 위젯들을 수직으로 배치할 수 있는 레이아웃 클래스(위젯 사이 간격, 여백 지정가능)
- ***QWidget:*** PyQt5의 모든 위젯의 기본 클래스(창이나 다이얼로그 등 인터페이스의 전반적인 틀로 사용)
- ***QPushButton:*** 버튼을 만드는 위젯으로, 클릭 시 특정 기능이나 이벤트를 실행(텍스트나, 이미지 추가 가능)
- ***QTimer:*** 일정 시간 간격으로 이벤트를 발생시키는 타이머 객체
- ***QImage:*** 이미지 데이터를 다루는 클래스(픽셀 단위로 조작하는 경우 유용,이미지 필터링이나 효과 처리에 사용)
- ***QPixmap:*** 이미지를 표시하기 위한 클래스(QLabel과 함께 사용하여 GUI에 이미지를 표시하며, QImage와 상호 변환이 가능)

## 5-7 ChatGPT와 FastAPI를 활용한 챗 서비스

- ChatGPT API를 사용해서 대화형 서비스 만들어보기
- 사용 프로그램: OpenAI, FastAPI


## 5-8 음성 생성과 번역을 활용한 번역 서비스


In [ ]:
# 5-6 실시간 객체 탐지 서비스

# 필수 라이브러리 임포트
from ultralytics import YOLO
import cv2
from PyQt5.QtWidgets import QApplication, QLabel, QVBoxLayout, QWidget, QPushButton
from PyQt5.QtCore import QTimer
from PyQt5.QtGui import QImage, QPixmap

# YOLOv8은 객체 탐지 모델
# cv2는 OpenCV의 라이브러리로 웹캠제어 및 영상처리
# PyQt5는 GUI를 생성하고 웹캠 시작 중지하는 버튼 추가

# VideoCaptureWidget클래스 생성
# PyQt5의 QWidget을 상속받아 GUI창으로 활용
class VideoCaptureWidget(QWidget):
    def __init__(self):
        super().__init__()

        # YOLOv8x 모델 로드 (YOLOv8x)
        self.model = YOLO('yolov8x.pt')
        # UI 설정
        self.setWindowTitle("실시간 객체 탐지")
        self.image_label = QLabel(self)
        self.layout = QVBoxLayout()
        self.layout.addWidget(self.image_label)

        self.start_button = QPushButton("Start Webcam", self)
        self.start_button.clicked.connect(self.start_webcam)
        self.layout.addWidget(self.start_button)

        self.stop_button = QPushButton("Stop Webcam", self)
        self.stop_button.clicked.connect(self.stop_webcam)
        self.layout.addWidget(self.stop_button)
        self.setLayout(self.layout)
      
        # 웹캠 초기화
        self.capture = None
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.update_frame)

    # 웹캠 시작
    def start_webcam(self):
        """웹캠을 시작하고, 타이머를 시작하여 프레임을 주기적으로 읽음"""
        self.capture = cv2.VideoCapture(0)  # 웹캠 장치 열기
        self.timer.start(20)  # 20ms마다 프레임 업데이트 (50fps)

    # 웹캠 중지
    def stop_webcam(self):
        """웹캠을 중지하고 타이머를 멈춤"""
        self.timer.stop()
        if self.capture is not None:
            self.capture.release()

    # 객체 탐지 및 UI업데이트        
    def update_frame(self):
        """웹캠에서 프레임을 읽어와서 YOLO 객체 탐지를 수행한 후 UI에 표시"""
        ret, frame = self.capture.read()
        if ret:
            # YOLOv8 객체 탐지 수행
            results = self.model(frame)
            result = results[0]

            # 바운딩 박스가 포함된 이미지를 가져옴
            img_with_boxes = result.plot()

            # OpenCV 이미지를 QImage로 변환
            rgb_image = cv2.cvtColor(img_with_boxes, cv2.COLOR_BGR2RGB)
            h, w, ch = rgb_image.shape
            bytes_per_line = ch * w
            convert_to_Qt_format = QImage(rgb_image.data, w, h, bytes_per_line, QImage.Format_RGB888)

            # QImage를 QLabel에 표시하기 위해 QPixmap으로 변환
            self.image_label.setPixmap(QPixmap.fromImage(convert_to_Qt_format))
            
    # 프로그램 종료 시 자원 해제
    def closeEvent(self, event):
        """윈도우 닫을 때 웹캠 해제"""
        if self.capture is not None:
            self.capture.release()
       
    # 메인 실행부
        if __name__ == "__main__":
            app = QApplication([])
            window = VideoCaptureWidget()
            window.show()
            app.exec_()
            

In [ ]:
# 5-7 ChatGPT와 FastAPI를 활용한 챗 서비스

from fastapi import FastAPI, Request, Form
from fastapi.templating import Jinja2Templates
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles
from openai import OpenAI

app = FastAPI()

# OpenAI API 클라이언트 설정
client = OpenAI()

# Jinja2 템플릿 설정
templates = Jinja2Templates(directory="app/templates")

# 정적 파일 서빙
app.mount("/static", StaticFiles(directory="app/static"), name="static")

# 초기 시스템 메시지 설정 (이 메시지는 사용자에게는 표시되지 않음)
system_message = {
    "role": "system",
    "content": "너는 환영 인사를 하는 인공지능이야, 농담을 넣어 재미있게해줘"
}

# 대화 내역을 저장할 리스트 초기화 (서버가 재시작되면 초기화됨)
messages = [system_message]

@app.get("/", response_class=HTMLResponse)
async def get_chat_page(request: Request):
    """채팅 페이지 렌더링"""
    # 대화 내역을 첫 로드 시에는 빈 리스트로 표시
    conversation_history = [msg for msg in messages if msg["role"] != "system"]
    return templates.TemplateResponse("index.html", {"request": request, "conversation_history": conversation_history})

@app.post("/chat", response_class=HTMLResponse)
async def chat(request: Request, user_input: str = Form(...)):
    """사용자 메시지를 받아 OpenAI API 호출 및 응답 반환"""
    global messages

    # 사용자의 메시지를 대화 내역에 추가
    messages.append({"role": "user", "content": user_input})

    # OpenAI API 호출
    completion = client.chat.completions.create(
        model="gpt-4o",  # 주신 코드에 맞춘 모델
        messages=messages
    )

    # AI의 응답 가져오기
    assistant_reply = completion.choices[0].message.content

    # AI의 응답을 대화 내역에 추가
    messages.append({"role": "assistant", "content": assistant_reply})

    # 화면에 표시할 대화 내역에서 system 메시지를 제외하고 전달
    conversation_history = [msg for msg in messages if msg["role"] != "system"]

    # 결과를 HTML로 반환 (대화 내역과 함께)
    return templates.TemplateResponse("index.html", {
        "request": request,
        "conversation_history": conversation_history
    })



In [ ]:
# 5-8 음성 생성과 번역을 활용한 번역 서비스
import os
import requests
from dotenv import load_dotenv
from PyQt5 import QtWidgets
from PyQt5.QtCore import QUrl
from PyQt5.QtMultimedia import QMediaPlayer, QMediaContent
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from pydub import AudioSegment
from pydub.playback import play
import io

# TranslatorApp클래스 생성
class TranslatorApp(QtWidgets.QWidget):
    def __init__(self):
        super().__init__()
        self.init_ui()

        # 번역 모델 로드
        model_name = "facebook/nllb-200-distilled-600M"
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

        # API 설정
        load_dotenv()
        self.api_key = os.getenv("API_KEY")
        self.url = os.getenv("API_URL")

        # 음성 재생기
        self.player = QMediaPlayer()

    # UI 구성
    def init_ui(self):
        # UI 구성
        self.text_input = QtWidgets.QLineEdit(self)
        self.text_input.setPlaceholderText("번역할 텍스트 입력")
        self.translate_button = QtWidgets.QPushButton("번역 및 음성 생성", self)
        self.output_label = QtWidgets.QLabel(self)
        self.play_button = QtWidgets.QPushButton("음성 재생", self)
        self.play_button.setEnabled(False)

        # 레이아웃 설정
        layout = QtWidgets.QVBoxLayout()
        layout.addWidget(self.text_input)
        layout.addWidget(self.translate_button)
        layout.addWidget(self.output_label)
        layout.addWidget(self.play_button)
        self.setLayout(layout)

        # 버튼 클릭 시 이벤트 핸들러 연결
        self.translate_button.clicked.connect(self.translate_and_generate_audio)
        self.play_button.clicked.connect(self.play_audio)

        # 윈도우 창 설정
        self.setWindowTitle("번역 및 음성 생성기")
        self.show()
   

    # 번역 및 음성 생성
    def translate_and_generate_audio(self):
        text = self.text_input.text()

        # 번역 수행
        inputs = self.tokenizer(text, return_tensors="pt")
        generated_tokens = self.model.generate(inputs.input_ids, forced_bos_token_id=self.tokenizer.lang_code_to_id["kor_Hang"])
        translated_text = self.tokenizer.decode(generated_tokens[0], skip_special_tokens=True)

        # 음성 생성 요청
        data = {
            "text": translated_text,
            "model_id": "eleven_multilingual_v2",
            "voice_settings": {
                "stability": 0.5,
                "similarity_boost": 1,
                "style": 0.5,
                "use_speaker_boost": True
            }
        }
        headers = {
            "xi-api-key": self.api_key,
            "Content-Type": "application/json"
        }
        response = requests.post(self.url, json=data, headers=headers)

        if response.status_code == 200:
            output_audio_path = "audio_output/output_audio.mp3"
            with open(output_audio_path, "wb") as f:
                f.write(response.content)

            self.output_label.setText(f"번역 결과: {translated_text}")
            self.play_button.setEnabled(True)
        else:
            self.output_label.setText("음성 생성 실패")

    # 음성 재생
    def play_audio(self):
        # 음성 파일 재생
        audio_path = "audio_output/output_audio.mp3"
        if os.path.exists(audio_path):
            # Pydub을 통해 mp3 파일을 불러와서 재생
            audio = AudioSegment.from_mp3(audio_path)
            play(audio)  # Pydub의 play() 함수 사용
        else:
            self.output_label.setText("오디오 파일을 찾을 수 없습니다.")

# 메인 실행            
if __name__ == '__main__':
    app = QtWidgets.QApplication([])
    translator = TranslatorApp()
    app.exec_()
            